In [9]:
import numpy as np
import random as rd

In [10]:
N=100
G=4.302*10**-3 #pc (km/s)^2 Msun^-1
size = 50
levels = int(np.floor(np.log(N)/np.log(4)))
eps=2
p = int(np.log2(eps))
numcells = 4**levels
rd.seed(10)

In [48]:
mass = np.ones((N)).reshape(-1,1)
x = [rd.uniform(-size,size) for i in range(N)]
y = [rd.uniform(-size,size) for i in range(N)]
points = np.array(list(zip(x,y)))
data = 1j*points[:,1]; data+=points[:,0]; data = data.reshape(N,1)

In [171]:
def cellNum(cells, coords): #Return the cell each particle belongs to, as well as the midpoint of the cell.
    cellbound = cells[::4]
    for i in range(len(cellbound)):
        if coords.real > cellbound[i]:
            indx = i
            break
    for j in range(len(cellbound)):
        if coords.imag > cellbound[j]:
            indy = j
            break
    return [indx,indy]

def cellMid(cells, inds): #Return the middle-point of the input cell
    cellmid = cells[2::4]
    midx = cellmid[inds[0]-1]
    midy = cellmid[inds[1]-1]
    return complex(midx, midy)

In [172]:
def childNum(cells, coords):
    cellbound = cells[::2]
    for i in range(len(cellbound)):
        if coords.real > cellbound[i]:
            indx = i
            break
    for j in range(len(cellbound)):
        if coords.imag > cellbound[j]:
            indy = j
            break
    return [indx,indy]
    
def childMid(cells, inds):
    cellmid = cells[1::2]
    midx = cellmid[inds[0]-1]
    midy = cellmid[inds[1]-1]
    return complex(midx, midy)

In [173]:
def coeff(z, q, k): #return expansion coefficients
    a = [-q[m]*(z[m]**k)/k for m in range(len(q))]
    return np.sum(a)

def potentialStep1(z,z0,q): #calculate potential centered around cell center, z0, out to p terms
    t1 = np.sum(q)*np.log(z0)
    t2 = np.sum([coeff(z-z0,q,k)/(z0**k) for k in range(1,p+1)])
    return t1+t2

def upPass1(data, mass): # Output (i x j x p) matrix
    cells = np.linspace(size,-size, 1+2**(levels+2)) #Mesh coordinates for finest mesh [level+2] (to make finding midpoints and children easier)
    cellnums = np.array([cellNum(cells, coords) for coords in data])#.reshape(N,1) #Calculate cell indices for each input point
    cellmids = np.array([cellMid(cells, inds) for inds in cellnums]) #Calculate cell mid-coordinates
    for i in range(1,2**levels+1):
        for j in range(1,2**levels+1):
            pointfilter = np.where( (cellnums[:,0] == i) & (cellnums[:,1] == j) )
            try:
                z0 = cellmids[pointfilter][0] #Mid coords for selected cell
                z = data[:,0][pointfilter] #Coords for points in selected cell
                q = mass[pointfilter] #Masses for selected points
                print(potentialStep1(z,z0,q)) #Potential of points around cell center ##This should be changed to be a matrix of expansion ;
                                                                    #coefficients that gets sent into the next step to be used to calculate coarser expansion potentials.
            except:
                continue
                
def upPass2(data, mass):
    l = levels - 1
    while(l >= 0):
        cells = np.linspace(size,-size,1+2**(l+2))
        cellnums = np.array([cellNum(cells, coords) for coords in data])
        cellmids = np.array([cellMid(cells, inds) for inds in cellnums])
        childnums = np.array([childNum(cells, coords) for coords in data])
        childmids = np.array([childMid(cells, inds) for inds in childnums])
        for i in range(1,2**l+1):
            for j in range(1,2**l+1):
                #Pull coefficients for the children of this cell, regardless of population.
                pointfilter = np.where( (cellnums[:,0] == i) & (cellnums[:,1] == j) )
                try:
                    z0 = cellmids[pointfilter][0] #Mid coords for selected cell
                    z = data[:,0][pointfilter] #Coords for points in selected cell
                    q = mass[pointfilter] #Masses for selected points
                    print(potentialStep1(z,z0,q)) #Potential of points around cell center
                except:
                    continue
        l-=1

In [199]:
test1 = np.linspace(size,-size, 1+2**(1+2)) #Mesh coordinates for finest mesh [level+2] (to make finding midpoints and children easier)
test2 = np.array([cellNum(test1, coords) for coords in data])#.reshape(N,1) #Calculate cell indices for each input point
test3 = np.array([cellMid(test1, inds) for inds in test2]).reshape(N,1) #Calculate cell mid-coordinates
test4 = np.array([childNum(test1, coords) for coords in data])
test5 = np.array([childMid(test1, inds) for inds in test4]).reshape(N,1)